In [ ]:
from lsts import LongTermForecast

ltf = LongTermForecast(
    pred_len=96, 
    variable="soil_temperature", 
    model_name="TimesNet"
)

In [ ]:
import pandas as pd
df = pd.read_csv("soil_temperature_example.csv")
df.rename(columns={"date_time": "date"}, inplace=True)
seq = df[:512].copy()
label = df[512: 512 + 96].copy()

In [ ]:
pred = ltf.pred(seq)

In [ ]:
pred

In [ ]:
ltf.visual(seq, pred, "soil_temperature", "soil_temperature_dlinear.pdf", label)

In [ ]:
label

In [ ]:
from lsts.short_term_forecast import ShortTermForecast

In [ ]:
stf = ShortTermForecast(
    variable="soil_temperature", 
    model_name="PatchTST"
)

In [ ]:
import pandas as pd
df = pd.read_csv("soil_temperature_example.csv")
df.rename(columns={"date_time": "date"}, inplace=True)
seq = df[:48].copy()
label = df[48: 48 + 48].copy()

In [ ]:
output = stf.pred(seq)

In [ ]:
stf.visual(seq, output, "soil_temperature", "stf_patchtst_soil_temperature.pdf", label)